In [2]:
import yfinance as yf
from datetime import datetime, timedelta
from get_all_tickers import get_tickers as gt
import seaborn as sns
import pandas as pd
import numpy as np
import os 

In [3]:
#useful tools 
today = (datetime.now() - timedelta(days=1) ).strftime('%Y-%m-%d')
month_1 = (datetime.now() - timedelta(days=31)).strftime('%Y-%m-%d')
year_1 = (datetime.now() - timedelta(days=366)).strftime('%Y-%m-%d')
year_5 = (datetime.now() - timedelta(days=(365 * 5  + 1))).strftime('%Y-%m-%d')
current_year = (datetime.now() - timedelta(days=1) ).strftime('%Y')

months = {
    'Jan': '01', 'Fev': '02', 'Mar': '03', 'Abr': '04',
    'Mai': '05', 'Jun': '06', 'Jul': '07', 'Ago': '08',
    'Set': '09', 'Out': '10', 'Nov': '11', 'Dez': '12'
}

# get all Brazilian tickers from stocks
br_df = pd.read_html("https://www.dadosdemercado.com.br/bolsa/acoes").copy()
tickers = br_df[0]['Ticker']
tickers = [tickers + ".SA" for tickers in tickers]



## Step 1: Collect and transforming the data from yfinance


In [4]:
big_data = yf.Ticker('ITUB4.SA')

stock_df = big_data.history(period='max' ).copy()
stock_df.drop('Stock Splits', inplace=True, axis=1)
stock_df['Yield'] = stock_df['Dividends'] / stock_df['Close']
stock_df = stock_df.astype('float')


In [5]:
stock_df


,Open,High,Low,Close,Volume,Dividends,Yield
Date,,,,,,,
2000-12-21 00:00:00-02:00,1.606090,1.704794,1.605901,1.688831,74224.0,0.00000,0.000000
2000-12-22 00:00:00-02:00,1.671829,1.671829,1.605712,1.615630,23312.0,0.00000,0.000000
2000-12-25 00:00:00-02:00,1.615630,1.615630,1.615630,1.615630,0.0,0.00000,0.000000
2000-12-26 00:00:00-02:00,1.606751,1.617046,1.591544,1.615158,7851.0,0.00000,0.000000
2000-12-27 00:00:00-02:00,1.643495,1.669942,1.615159,1.669942,26996.0,0.00000,0.000000
...,...,...,...,...,...,...,...
2023-12-28 00:00:00-03:00,33.862397,33.952351,33.722471,33.952351,12555300.0,0.00000,0.000000
2024-01-02 00:00:00-03:00,33.889999,33.889999,33.240002,33.520000,18948200.0,0.01765,0.000527
2024-01-03 00:00:00-03:00,33.509998,33.770000,33.150002,33.150002,18530900.0,0.00000,0.000000


## Step 2: Get data all from inflation and CDI CSV file


In [6]:
FILE_FOLDER = os.path.abspath('')
rates_df =  pd.read_csv(os.path.join(FILE_FOLDER, 'Inflation_CDI_data'))



In [7]:
# convert rate_df into a time series
rates_df['Unnamed: 0'] = pd.to_datetime(rates_df['Unnamed: 0'], utc=True).dt.tz_convert('America/Sao_Paulo')
rates_df.set_index('Unnamed: 0', inplace=True)



In [8]:
# fill rates df with days 
rates_df = rates_df.resample('D').ffill()
rates_df

,Inflation,CDI
Unnamed: 0,,
2000-01-31 00:00:00-02:00,NaN,NaN
2000-02-01 00:00:00-02:00,0.13,1.4405
2000-02-02 00:00:00-02:00,0.13,1.4405
2000-02-03 00:00:00-02:00,0.13,1.4405
2000-02-04 00:00:00-02:00,0.13,1.4405
...,...,...
2023-11-26 00:00:00-03:00,0.28,0.9160
2023-11-27 00:00:00-03:00,0.28,0.9160
2023-11-28 00:00:00-03:00,0.28,0.9160


In [9]:
# remove non days of week
rates_df = rates_df[rates_df.index.isin(stock_df.index)]

In [10]:
# convert monthly rates into daily rates
rates_df = rates_df / 100

In [11]:
# merge the stock and rates df 
df = pd.merge(rates_df, stock_df, left_index=True,right_index=True, how='outer')

In [12]:
# fill missing inflation and cdi with the last most recent data
df['CDI'].ffill(inplace=True)
df['Inflation'].ffill(inplace=True)

In [13]:
# convert inflation and CDI rates to daily
df['CDI'] = (1 + df['CDI']) ** (1/20) - 1
df['Inflation'] = (1 + df['Inflation']) ** (1/20) - 1

In [14]:
# create return values to stock, inflation and CDI 
df['Return_Stock'] = (df['Close'] + df['Yield'] - df['Close'].shift(1) ) / df['Close'].iloc[0] 
df['Return_CDI'] = (df['Close'].iloc[0] * df['CDI'] )  / df['Close'].iloc[0]
df['Return_Inflation'] = (df['Close'].iloc[0] * df['Inflation'] )  / df['Close'].iloc[0]
df

,Inflation,CDI,Open,High,Low,Close,Volume,Dividends,Yield,Return_Stock,Return_CDI,Return_Inflation
2000-12-21 00:00:00-02:00,0.000294,0.000594,1.606090,1.704794,1.605901,1.688831,74224.0,0.00000,0.000000,NaN,0.000594,0.000294
2000-12-22 00:00:00-02:00,0.000294,0.000594,1.671829,1.671829,1.605712,1.615630,23312.0,0.00000,0.000000,-0.043344,0.000594,0.000294
2000-12-25 00:00:00-02:00,0.000294,0.000594,1.615630,1.615630,1.615630,1.615630,0.0,0.00000,0.000000,0.000000,0.000594,0.000294
2000-12-26 00:00:00-02:00,0.000294,0.000594,1.606751,1.617046,1.591544,1.615158,7851.0,0.00000,0.000000,-0.000280,0.000594,0.000294
2000-12-27 00:00:00-02:00,0.000294,0.000594,1.643495,1.669942,1.615159,1.669942,26996.0,0.00000,0.000000,0.032439,0.000594,0.000294
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-28 00:00:00-03:00,0.000140,0.000456,33.862397,33.952351,33.722471,33.952351,12555300.0,0.00000,0.000000,0.053264,0.000456,0.000140
2024-01-02 00:00:00-03:00,0.000140,0.000456,33.889999,33.889999,33.240002,33.520000,18948200.0,0.01765,0.000527,-0.255694,0.000456,0.000140
2024-01-03 00:00:00-03:00,0.000140,0.000456,33.509998,33.770000,33.150002,33.150002,18530900.0,0.00000,0.000000,-0.219086,0.000456,0.000140
2024-01-04 00:00:00-03:00,0.000140,0.000456,33.150002,33.230000,32.700001,32.930000,26585200.0,0.00000,0.000000,-0.130268,0.000456,0.000140


In [15]:
df['Return_Stock'] = df['Return_Stock'].cumsum()
df['Return_CDI'] = df['Return_CDI'].cumsum()
df['Return_Inflation'] = df['Return_Inflation'].cumsum()


In [16]:
# # create moving average for candlestick graph
# df['MMS15'] = df['Close'].rolling(window = 15).mean()
# df['MME15'] = df['Close'].ewm(span = 15).mean()

In [17]:
# show the final result
df

,Inflation,CDI,Open,High,Low,Close,Volume,Dividends,Yield,Return_Stock,Return_CDI,Return_Inflation,MMS15,MME15
2000-12-21 00:00:00-02:00,0.000294,0.000594,1.606090,1.704794,1.605901,1.688831,74224.0,0.00000,0.000000,NaN,0.000594,0.000294,NaN,1.688831
2000-12-22 00:00:00-02:00,0.000294,0.000594,1.671829,1.671829,1.605712,1.615630,23312.0,0.00000,0.000000,-0.043344,0.001187,0.000588,NaN,1.649790
2000-12-25 00:00:00-02:00,0.000294,0.000594,1.615630,1.615630,1.615630,1.615630,0.0,0.00000,0.000000,-0.043344,0.001781,0.000883,NaN,1.636854
2000-12-26 00:00:00-02:00,0.000294,0.000594,1.606751,1.617046,1.591544,1.615158,7851.0,0.00000,0.000000,-0.043624,0.002374,0.001177,NaN,1.630300
2000-12-27 00:00:00-02:00,0.000294,0.000594,1.643495,1.669942,1.615159,1.669942,26996.0,0.00000,0.000000,-0.011184,0.002968,0.001471,NaN,1.640473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-28 00:00:00-03:00,0.000140,0.000456,33.862397,33.952351,33.722471,33.952351,12555300.0,0.00000,0.000000,19.549016,2.717553,1.446659,32.732984,32.766735
2024-01-02 00:00:00-03:00,0.000140,0.000456,33.889999,33.889999,33.240002,33.520000,18948200.0,0.01765,0.000527,19.293322,2.718009,1.446799,32.860746,32.860893
2024-01-03 00:00:00-03:00,0.000140,0.000456,33.509998,33.770000,33.150002,33.150002,18530900.0,0.00000,0.000000,19.074236,2.718465,1.446939,32.937188,32.897032
2024-01-04 00:00:00-03:00,0.000140,0.000456,33.150002,33.230000,32.700001,32.930000,26585200.0,0.00000,0.000000,18.943968,2.718921,1.447079,33.013623,32.901153


In [18]:
def transform_data(stock_df_name, per='None', start=None, end=None):
    # get all major data from yahoo finance
    big_data = yf.Ticker(stock_df_name)

    stock_df = big_data.history(period=per, start=start, end=end).copy()
    columns = ["Close", 'Volume', 'Open', 'High', 'Low', 'Dividends']
    stock_df = pd.DataFrame(stock_df[columns])
    stock_df['Yield'] = stock_df['Dividends'] / stock_df['Close']
    stock_df = stock_df.astype('float')
    
    # get csv file from data_storage

    FILE_FOLDER = os.path.abspath('')
    rates_df =  pd.read_csv(os.path.join(FILE_FOLDER, 'Inflation_CDI_data'))
    #transform rates_df into time series
    rates_df['Unnamed: 0'] = pd.to_datetime(rates_df['Unnamed: 0'], utc=True).dt.tz_convert('America/Sao_Paulo')
    rates_df.set_index('Unnamed: 0', inplace=True)

    # fill rates_df with days 
    rates_df = rates_df.resample('D').ffill()

    # remove non days of week and resize rates to percentage
    rates_df = rates_df[rates_df.index.isin(stock_df.index)]
    rates_df = rates_df / 100




    # merge dfs and fill with days of week
    df = pd.merge(rates_df, stock_df, left_index=True,right_index=True, how='outer')
    df['CDI'].ffill(inplace=True)
    df['Inflation'].ffill(inplace=True)

    # convert inflation and CDI rates to daily
    df['CDI'] = (1 + df['CDI']) ** (1/20) - 1
    df['Inflation'] = (1 + df['Inflation']) ** (1/20) - 1

    # create return values to stock, inflation and CDI 
    df['Return_Stock'] = (df['Close'] + df['Yield'] - df['Close'].shift(1) ) / df['Close'].iloc[0] 
    df['Return_CDI'] = (df['Close'].iloc[0] * df['CDI'] )  / df['Close'].iloc[0]
    df['Return_Inflation'] = (df['Close'].iloc[0] * df['Inflation'] )  / df['Close'].iloc[0]
    
    # transform above columns values into cumulative sum
    df['Return_Stock'] = df['Return_Stock'].cumsum()
    df['Return_CDI'] = df['Return_CDI'].cumsum()
    df['Return_Inflation'] = df['Return_Inflation'].cumsum()
    # create moving average for candlestick graph
    # df['MMS15'] = df['Close'].rolling(window = 15).mean()
    # df['MME15'] = df['Close'].ewm(span = 15).mean()

    # avoid missing values

    df = df.iloc[1:]
    return df



In [33]:
# final df
month_6 = (datetime.now() - timedelta(days=31 * 8)).strftime('%Y-%m-%d')
df = transform_data("ITUB4.SA", start=month_6)


In [35]:
# condition for show candle graph without lost visualization
if len(df) < 125:
    candle_df = df
elif len(df) > 125 and len(df) < 247 :
    candle_df = df.resample('W-Mon').first()
else:
    candle_df = df.resample('MS').first()